In [1]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import os
import cv2
import tifffile as tiff
import segmentation_models as sm
import tensorflow.image as tfi
from PIL import Image
tf.debugging.set_log_device_placement(True)
from tensorflow import keras
# import segmentation_models as sm
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers,models, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, SpatialDropout2D, BatchNormalization, Dropout
from tensorflow.keras.layers import Lambda
from pathlib import Path
from tensorflow.keras.utils import load_img, img_to_array, plot_model
from tensorflow.keras.regularizers import l2
from sklearn.decomposition import PCA
from tensorflow.keras.applications import VGG16
# from keras_segmentation.models.unet import vgg_unet
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose

Segmentation Models: using `tf.keras` framework.


In [2]:
def load_images_pillow(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if "_" not in filename:
            if filename.endswith('.png') or filename.endswith('.jpg'):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path)
                img_array = np.array(img)
                images.append(img_array)
    return images

In [3]:
def convert_to_pca(image):
    images = image.reshape(-1, 12)
    images = np.array(images, dtype=np.float32)
    
    pca = PCA(n_components=3)
    images_pca = pca.fit_transform(images)
    images = images_pca.reshape(128, 128, 3)
    images = (images - np.min(images)) / (np.max(images) - np.min(images))
    return images

In [4]:
def conv_block(inputs, filters, kernel=3, activation='relu', padding='same' ,kernel_initializer='he_normal'):
    conv1 = Conv2D(filters, kernel, activation=activation, padding=padding ,kernel_initializer=kernel_initializer)(inputs)
    b1 = BatchNormalization()(conv1)
    s1 = SpatialDropout2D(.5)(b1)
    conv2 = Conv2D(filters, kernel, activation=activation, padding=padding ,kernel_initializer=kernel_initializer)(s1)
    s2 = SpatialDropout2D(.5)(conv2)
    b2 = BatchNormalization()(s2)

    
    return b2

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

def convert_12_to_3_channels_with_conv2d(images):
    # Define the Conv2D layer to reduce the number of channels from 12 to 3
    conv_layer = layers.Conv2D(filters=3, kernel_size=(1, 1), padding='same')

    # Apply the Conv2D layer
    output_tensor = conv_layer(images)

    # Convert the output to a Keras tensor (though it's already a tensor by default)
    output_tensor = tf.convert_to_tensor(output_tensor, dtype= tf.float32)

    return output_tensor

# Example usage:
# Assuming input_tensor is your (batch_size, height, width, 12) image tensor
# output_tensor = convert_12_to_3_channels_with_conv2d(input_tensor)


In [6]:
def sampling_block(inputs, pre, filters, kernel=3, activation='relu', padding='same' ,kernel_initializer='he_normal'):
    up1 = UpSampling2D()(inputs)
    conv1 = Conv2DTranspose(filters, kernel, activation=activation, padding=padding ,kernel_initializer=kernel_initializer)(up1)
    merge1 = concatenate([pre, conv1])
    
    conv2 = Conv2D(filters, kernel, activation=activation, padding=padding ,kernel_initializer=kernel_initializer)(merge1)
    s1 = Dropout(0.2)(conv2)
    b2 = BatchNormalization()(s1)

    conv3 = Conv2D(filters, kernel, activation=activation, padding=padding ,kernel_initializer=kernel_initializer)(s1)
    s2 = Dropout(0.2)(conv3)
    b3 = BatchNormalization()(s2)
    
    return b3

In [7]:
def unet_model(n_filters=64, input_size= (128,128,12)):
    
    input_layer = Input(shape=(128, 128, 12))
    
    # Encoder
    out1 = conv_block(input_layer, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(out1)

    out2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(out2)

    out3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(out3)

    out4 = conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(out4)
    
    #Bottleneck
    out5 = conv_block(pool4, 1024)
#     pool5 = MaxPooling2D(pool_size=(2, 2))(out5)

    
    #Decoder
    sam1 = sampling_block(out5, out4, 512)

    sam2 = sampling_block(sam1, out3, 256)

    sam3 = sampling_block(sam2, out2, 128)

    sam4 = sampling_block(sam3, out1, 64)  
    
    final_conv = Conv2D(1, (1, 1), activation='sigmoid')(sam4)

    model = Model(inputs=input_layer, outputs=final_conv)

    return model

In [8]:
import tensorflow as tf
import numpy as np

def augment_image_mask(image, mask):
    seed = np.random.randint(0, 10000)
    
    # Random Flip (horizontal and vertical)
    aug_img = tf.image.random_flip_left_right(image, seed=seed)
    aug_mask = tf.image.random_flip_left_right(mask, seed=seed)
    
    aug_img = tf.image.random_flip_up_down(aug_img, seed=seed)
    aug_mask = tf.image.random_flip_up_down(aug_mask, seed=seed)
    
    # Random Rotation (up to 20 degrees)
    angle = np.random.uniform(-20, 20)
    angle_rad = angle * np.pi / 180  # Convert to radians
    aug_img = tf.image.rot90(aug_img, k=np.random.randint(0, 4))  # Rotate 0, 90, 180, or 270 degrees
    aug_mask = tf.image.rot90(aug_mask, k=np.random.randint(0, 4))
    
    # Shearing (using affine transformation)
    shear_factor = np.random.uniform(-0.2, 0.2)
    transformation_matrix = [[1, shear_factor, 0], [0, 1, 0], [0, 0, 1]]
    aug_img = tf.keras.preprocessing.image.apply_affine_transform(aug_img.numpy(), shear=shear_factor)
    aug_mask = tf.keras.preprocessing.image.apply_affine_transform(aug_mask.numpy(), shear=shear_factor)
    
    return aug_img, aug_mask


In [51]:
images_path = "/kaggle/input/water-segmentation/data/images/*.tif"
labels_path = "/kaggle/input/water-segmentation/data/labels"

images = tiff.imread(images_path)
labels = load_images_pillow(labels_path)

In [52]:
for i in range(12):
    images[:, :,:,i] = images[:, :,:,i] / images[:, :,:,i].max()

In [53]:
X_train, X_valid, y_train, y_valid = train_test_split(images, labels, test_size= .1, random_state= 42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size= .1, random_state= 42)

In [ ]:
for i in range(len(X_train)):
    X_train[i], y_train[i] = augment_image_mask(X_train[i], tf.expand_dims(y_train[i] , axis= -1))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
shape: (_DeviceArg): /job:localhost/replica:0/task:0/device:CPU:0
RandomUniform: (RandomUniform): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Execut

In [ ]:
y_train = tf.cast(y_train, axis= -1, dtype= tf.float32)
y_valid= tf.expand_dims(y_valid , axis= -1, dtype= tf.float32)
y_test= tf.expand_dims(y_test , axis= -1, dtype= tf.float32)

In [ ]:
# y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

# y_valid = tf.convert_to_tensor(y_valid, dtype=tf.float32)

# y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


In [ ]:
# Create checkpoint callback
checkpoint_path = "vgg_unet.keras"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 10,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [ ]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Apply the generator to your training data
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

In [ ]:
steps_per_epoch = len(X_train) // 32
validation_steps = len(X_valid) // 32

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# preprocess input
X_train = preprocess_input(X_train)
X_valid = preprocess_input(X_valid)

# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(optimizer=tf.keras.optimizers.Adam(.01),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

model.fit(
    x=X_train,
    y=tf.expand_dims(y_train, axis= -1),
    batch_size=16,
    epochs=100,
    validation_data=(X_valid, y_valid),
    steps_per_epoch = steps_per_epoch   
)

In [ ]:
results = u_net_model.evaluate(test_generator , verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# Trash


In [ ]:
# history = model.fit(
#     train_generator,
#     epochs=50,
#     validation_data= valid_generator,
#     steps_per_epoch=steps_per_epoch,
#     validation_steps=validation_steps,
#     callbacks=[
#         early_stopping,
#         reduce_lr, 
#         checkpoint_callback
#     ]
#  )

In [ ]:
# def read_label(path, size= 128):
#     label = load_img(path)
# #     label = img_to_array(label)
# #     label = cv2.cvtColor(label, cv2.COLOR_RGB2GRAY)
#     label = tf.convert_to_tensor(label, dtype= tf.float32)
# #     label = tf.cast(label, tf.float32)
#     label *= 255.
#     label = tf.convert_to_tensor(label, dtype= tf.float32)
#     return label

In [ ]:
#     inputs = tf.keras.layers.Input((128,128,3))
    
#     cblock1 = conv_block(inputs, num_filters= n_filters)
#     cblock2 = conv_block(cblock1[0], num_filters= n_filters*2)
#     cblock3 = conv_block(cblock2[0], num_filters= n_filters*4)
#     cblock4 = conv_block(cblock3[0], num_filters= n_filters*8)
#     cblock5 = conv_block(cblock4[0], num_filters= n_filters*16, Max_pool=0)
    
    
#     ublock6 = upsampling_block(cblock5[0], cblock4[1],  n_filters * 8)
#     ublock7 = upsampling_block(ublock6, cblock3[1],  n_filters * 4)
#     ublock8 = upsampling_block(ublock7, cblock2[1],  n_filters * 2)
#     ublock9 = upsampling_block(ublock8, cblock1[1],  n_filters)

#     conv9 = tf.keras.layers.Conv2D(n_filters,3,
#                  activation='relu',
#                  padding='same',
#                  kernel_initializer='he_normal')(ublock9)
    
    
#     conv10 = tf.keras.layers.Conv2D(3, kernel_size=3, padding="same",activation = 'sigmoid')(conv9)
#     model = tf.keras.Model(inputs=inputs, outputs=conv10)
#     return model

In [ ]:
# input_folder = "/kaggle/input/water-segmentation/data/labels"
# output_folder = '/kaggle/working/labels'
# os.makedirs(output_folder, exist_ok=True)

# for filename in os.listdir(input_folder):
#     if filename.endswith('.png') or filename.endswith('.jpg'):
#         # Open image file
#         img = Image.open(os.path.join(input_folder, filename))
#         # Convert image to grayscale
#         gray_img = img.convert('L')
#         # Save image with new dimensions
#         gray_img = gray_img.resize((128, 128))
#         gray_img = gray_img.convert('L')  # Ensure it's grayscale
#         gray_img.save(os.path.join(output_folder, filename))

In [ ]:
# images = tiff.imread(images_path)
# # labels = read_label(os.path.join(labels_path, x)) for x in os.listdir(labels_path) if "_" not in x
# i = 0
# labels 
# for x in os.listdir(labels_path):
#     if "_" not in x:
#         labels[i] = 

In [ ]:
# import numpy as np
# from tensorflow.image import resize

# def data_generator(images, labels, batch_size, target_size=(128, 128)):
#     while True:
#         for start in range(0, len(images), batch_size):
#             end = min(start + batch_size, len(images))
#             X_batch = images[start:end]
#             y_batch = labels[start:end]

#             # Resize images and labels to the target size
#             X_batch_resized = np.array([resize(img, target_size) for img in X_batch])
#             y_batch_resized = np.array([resize(lbl, target_size) for lbl in y_batch])

#             # Normalize images
#             X_batch_resized = X_batch_resized / 255.0

#             # Ensure labels have the correct shape (e.g., for binary segmentation)
#             if y_batch_resized.ndim == 4 and y_batch_resized.shape[-1] != 1:
#                 y_batch_resized = np.expand_dims(y_batch_resized[..., 0], axis=-1)

#             yield X_batch_resized, y_batch_resized


In [ ]:
# train_generator = train_datagen.flow(
#     x= X_train,
#     y= y_train,
#     batch_size=32,
# )

# valid_generator = train_datagen.flow(
#     x= X_valid,
#     y= y_valid,
#     batch_size=32,
# )

# test_datagen_generator = train_datagen.flow(
#     x= X_test,
#     y= y_test,
#     batch_size=32,
#     shuffle= False
# )

In [ ]:
# labels = np.array(labels, dtype=np.float32)
# labels = np.array([cv2.cvtColor(label, cv2.COLOR_RGB2GRAY) for label in labels])
# for label in range(len(labels)):
#     labels[label] = np.array(labels[label], dtype=np.float32)
#     labels[label] = cv2.cvtColor(labels[label], cv2.COLOR_RGB2GRAY)
    

# # Optionally, add a channel dimension back if needed (to match the U-Net input/output shape)
# labels = np.expand_dims(labels, axis=-1)

In [ ]:
# batch_size = 32
# target_size = (128, 128)  # Make sure this matches the output size of your model

# # Create generators
# train_generator = train_generator(X_train, y_train, batch_size, target_size)
# val_generator = val_generator(X_valid, y_valid, batch_size, target_size)
# test_generator = test_generator(X_test, y_test, batch_size, target_size)

In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# val_datagen = ImageDataGenerator(rescale=1./255)

# test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# labels= tf.convert_to_tensor(labels)
# images= tf.convert_to_tensor(images)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.applications import VGG16

# def vgg_unet_12_channel(input_shape=(128, 128, 3), num_classes=1):
    
#     inputs = layers.Input(shape=input_shape)
    
# #     # Use a Conv2D layer to reduce 12 channels to 3 channels
# #     conv_input = layers.Conv2D(3, (1, 1), padding='same', activation='relu')(inputs)
    
#     # Load the VGG16 model without the top layers (FC layers) and no input shape specification
#     vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# #     # Pass conv_input through each layer manually (don't specify input_tensor to avoid conflicts)
# #     x = conv_input
# #     for layer in vgg_base.layers:
# #         x = layer(x)
    
#     # Encoder - Extract intermediate layers from VGG for skip connections
#     block1_conv2 = vgg_base.get_layer("block1_conv2").output  # (128, 128, 64)
#     block2_conv2 = vgg_base.get_layer("block2_conv2").output  # (64, 64, 128)
#     block3_conv3 = vgg_base.get_layer("block3_conv3").output  # (32, 32, 256)
#     block4_conv3 = vgg_base.get_layer("block4_conv3").output  # (16, 16, 512)
#     block5_conv3 = vgg_base.get_layer("block5_conv3").output  # (8, 8, 512)

#     # Decoder
#     # Upsample block 5
#     up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding="same")(block5_conv3)  # (16, 16, 512)
#     up6 = layers.concatenate([up6, block4_conv3])  # Skip connection
#     up6 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(up6)
#     up6 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(up6)

#     # Upsample block 4
#     up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(up6)  # (32, 32, 256)
#     up7 = layers.concatenate([up7, block3_conv3])  # Skip connection
#     up7 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(up7)
#     up7 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(up7)

#     # Upsample block 3
#     up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(up7)  # (64, 64, 128)
#     up8 = layers.concatenate([up8, block2_conv2])  # Skip connection
#     up8 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(up8)
#     up8 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(up8)

#     # Upsample block 2
#     up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(up8)  # (128, 128, 64)
#     up9 = layers.concatenate([up9, block1_conv2])  # Skip connection
#     up9 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(up9)
#     up9 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(up9)

#     # Output layer
#     outputs = layers.Conv2D(num_classes, (1, 1), activation="sigmoid")(up9)  # (128, 128, num_classes)

#     # Final model
#     model = models.Model(inputs=inputs, outputs=outputs)

#     return model

# # Example Usage
# input_shape = (128, 128, 3)  # For 12-channel images
# num_classes = 1  # Binary segmentation
# model = vgg_unet_12_channel(input_shape, num_classes)

# # Compile the model
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# # Summary of the model
# model.summary()


In [ ]:
# def vgg_conv_block(inputs, filters, kernel=3, activation='relu', padding='same' ,kernel_initializer='he_normal'):
    
# #     inputs= layers.Input(shape=(128, 128, 12))
#     b0 = BatchNormalization()(inputs)
#     conv0 = Conv2D(3, (1, 1), activation="relu", padding="same")(inputs)
#     vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3), input_tensor= b0)(b0)
#     vgg_base.trainable = False

# #     b1 = BatchNormalization()(vgg_base)
# #     s1 = SpatialDropout2D(.2)(b1)
# #     conv2 = Conv2D(filters, kernel, activation=activation, padding=padding ,kernel_initializer=kernel_initializer)(s1)
# #     s2 = SpatialDropout2D(.2)(conv2)
# #     b2 = BatchNormalization()(s2)
    
    
#     return b2

In [ ]:
# def unet_model(n_filters=64, input_size= (128,128,12)):
    
#     input_layer = Input(shape=(128, 128, 12))
#     conv0 = Conv2D(3, (1,1), activation="relu", padding="same")(input_layer)


#     # VGG Encoder
#     vgg = VGG16(include_top=False, weights='imagenet', input_tensor=conv0)
#     vgg.trainable = False  # Freeze the VGG layers to keep ImageNet weights

#     # Encoder layers
#     block1_conv2 = vgg.get_layer("block1_conv2").output  # (None, 128, 128, 64)
#     block2_conv2 = vgg.get_layer("block2_conv2").output  # (None, 64, 64, 128)
#     block3_conv3 = vgg.get_layer("block3_conv3").output  # (None, 32, 32, 256)
#     block4_conv3 = vgg.get_layer("block4_conv3").output  # (None, 16, 16, 512)
#     block5_conv3 = vgg.get_layer("block5_conv3").output

    
#     #Decoder
#     sam1 = sampling_block(block5_conv3, block4_conv3, 512)

#     sam2 = sampling_block(sam1, block3_conv3, 256)

#     sam3 = sampling_block(sam2, block2_conv2, 128)

#     sam4 = sampling_block(sam3, block1_conv2, 64)  
    
#     final_conv = Conv2D(1, (1, 1), activation='sigmoid')(sam4)

#     model = Model(inputs=input_layer, outputs=final_conv)

#     return model

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.applications import VGG16

# def vgg_unet_12_channel(input_shape=(128, 128, 12), num_classes=1):
#     """
#     VGG-based U-Net model for 12-channel input image segmentation.
    
#     Parameters:
#     - input_shape: Tuple representing input image size (H, W, 12)
#     - num_classes: Number of output channels (1 for binary segmentation)
    
#     Returns:
#     - model: TensorFlow Keras model
#     """
    
#     # Input layer for 12-channel input
#     inputs = layers.Input(shape=input_shape)
    
#     # Use a Conv2D layer to reduce 12 channels to 3 channels
#     conv_input = layers.Conv2D(3, (1, 1), padding='same', activation='relu')(inputs)
    
#     # Load the VGG16 model with weights pre-trained on ImageNet and no top layer
#     vgg_base = VGG16(weights='imagenet', include_top=False, input_tensor=conv_input)

#     # Encoder - Extract intermediate layers from VGG for skip connections
#     block1_conv2 = vgg_base.get_layer("block1_conv2").output  # (128, 128, 64)
#     block2_conv2 = vgg_base.get_layer("block2_conv2").output  # (64, 64, 128)
#     block3_conv3 = vgg_base.get_layer("block3_conv3").output  # (32, 32, 256)
#     block4_conv3 = vgg_base.get_layer("block4_conv3").output  # (16, 16, 512)
#     block5_conv3 = vgg_base.get_layer("block5_conv3").output  # (8, 8, 512)

#     # Decoder
#     # Upsample block 5
#     up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding="same")(block5_conv3)  # (16, 16, 512)
#     up6 = layers.concatenate([up6, block4_conv3])  # Skip connection
#     up6 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(up6)
#     up6 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(up6)

#     # Upsample block 4
#     up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(up6)  # (32, 32, 256)
#     up7 = layers.concatenate([up7, block3_conv3])  # Skip connection
#     up7 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(up7)
#     up7 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(up7)

#     # Upsample block 3
#     up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(up7)  # (64, 64, 128)
#     up8 = layers.concatenate([up8, block2_conv2])  # Skip connection
#     up8 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(up8)
#     up8 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(up8)

#     # Upsample block 2
#     up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(up8)  # (128, 128, 64)
#     up9 = layers.concatenate([up9, block1_conv2])  # Skip connection
#     up9 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(up9)
#     up9 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(up9)

#     # Output layer
#     outputs = layers.Conv2D(num_classes, (1, 1), activation="sigmoid")(up9)  # (128, 128, num_classes)

#     # Final model
#     model = models.Model(inputs=inputs, outputs=outputs)

#     return model

# # Example Usage
# input_shape = (128, 128, 12)  # For 12-channel images
# num_classes = 1  # Binary segmentation
# model = vgg_unet_12_channel(input_shape, num_classes)

# # Load weights (only for VGG layers, skipping top layers)
# # vgg_weights_path = 'path_to_vgg16_weights.h5'
# # vgg_base = VGG16(weights=None, include_top=False, input_shape=(128, 128, 3))
# # vgg_base.load_weights(vgg_weights_path, by_name=True)  # Load weights by name to avoid layer count issues

# # Compile the model
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# # Summary of the model
# model.summary()


In [ ]:
# batch_size = 32
# target_size = (128, 128)

# # Convert numpy arrays to TensorFlow datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# val_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# # Shuffle, batch, and prefetch the datasets
# batch_size = 32
# train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
# val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
# test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)